In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
# First, enter the following code to download a Postgres driver that will allow Spark to interact with Postgres:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-07-30 21:27:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.64MB/s    in 0.2s    

2021-07-30 21:27:44 (5.64 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final_data").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col, avg, sum, count, rand, randn, to_timestamp

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://tq2cutebucket.s3.us-west-1.amazonaws.com/cali_house_sales.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("cali_house_sales.csv"), sep=",", header=True, inferSchema=True)
df.show()

+------+--------------------+--------+----------+----------+---+-----------+--------+---+---------------+--------------+--------------+-------------------+----------+--------------+---------+
|Status|      Street Address|Zip Code|List Price|Sale Price|Age|Sq Ft Total|Lot Size|DOM|   Listing Date|     Sale Date|    Close Date|Original List Price|Year Built|    MLS Number|Year_Sale|
+------+--------------------+--------+----------+----------+---+-----------+--------+---+---------------+--------------+--------------+-------------------+----------+--------------+---------+
|  Sold|    6882 Norton AVE |   93003|    465000|    415000| 63|       1907|  8400.0| 61| 11/5/2018 0:00| 1/5/2019 0:00|1/18/2019 0:00|             499900|      1958|CRV0-218013798|     2019|
|  Sold| 3177 Wood Creek DR |   95928|    475000|    475000| 21|       1800|  4792.0|  5|  1/2/2019 0:00| 1/7/2019 0:00|1/28/2019 0:00|             475000|      2000|   ME201900019|     2019|
|  Sold|     3011 S 5th AVE |   95965|  

In [ ]:
# Rename Zip Code in capital to zip_code
df = df.withColumnRenamed("Zip Code", "zip_code")
#.withColumnRenamed("colName2", "newColName2")
df.show()

+------+--------------------+--------+----------+----------+---+-----------+--------+---+---------------+--------------+--------------+-------------------+----------+--------------+---------+
|Status|      Street Address|zip_code|List Price|Sale Price|Age|Sq Ft Total|Lot Size|DOM|   Listing Date|     Sale Date|    Close Date|Original List Price|Year Built|    MLS Number|Year_Sale|
+------+--------------------+--------+----------+----------+---+-----------+--------+---+---------------+--------------+--------------+-------------------+----------+--------------+---------+
|  Sold|    6882 Norton AVE |   93003|    465000|    415000| 63|       1907|  8400.0| 61| 11/5/2018 0:00| 1/5/2019 0:00|1/18/2019 0:00|             499900|      1958|CRV0-218013798|     2019|
|  Sold| 3177 Wood Creek DR |   95928|    475000|    475000| 21|       1800|  4792.0|  5|  1/2/2019 0:00| 1/7/2019 0:00|1/28/2019 0:00|             475000|      2000|   ME201900019|     2019|
|  Sold|     3011 S 5th AVE |   95965|  

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://tq2cutebucket.s3.us-west-1.amazonaws.com/county_zipcode.csv"
spark.sparkContext.addFile(url)
county_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("county_zipcode.csv"), sep=",", header=True, inferSchema=True)
county_df.show()
county_df.count()

+--------+--------------+
|zip_code|        county|
+--------+--------------+
|   94063|     San Mateo|
|   92225|     Riverside|
|   92273|      Imperial|
|   92651|        Orange|
|   92801|        Orange|
|   92865|        Orange|
|   92352|San Bernardino|
|   92250|      Imperial|
|   92807|        Orange|
|   92627|        Orange|
|   91935|     San Diego|
|   91773|   Los Angeles|
|   91107|   Los Angeles|
|   93283|          Kern|
|   92866|        Orange|
|   94528|  Contra Costa|
|   95076|    Santa Cruz|
|   90010|   Los Angeles|
|   91905|     San Diego|
|   91016|   Los Angeles|
+--------+--------------+
only showing top 20 rows



51011

In [ ]:
# Join the two DataFrame
joined_df = df.join(county_df, on="zip_code", how="inner")
joined_df.show()
joined_df.count()

+--------+------+--------------------+----------+----------+---+-----------+--------+---+--------------+--------------+--------------+-------------------+----------+------------+---------+--------------+
|zip_code|Status|      Street Address|List Price|Sale Price|Age|Sq Ft Total|Lot Size|DOM|  Listing Date|     Sale Date|    Close Date|Original List Price|Year Built|  MLS Number|Year_Sale|        county|
+--------+------+--------------------+----------+----------+---+-----------+--------+---+--------------+--------------+--------------+-------------------+----------+------------+---------+--------------+
|   94063|  Sold|    757 Douglas AVE |    979998|   1350000| 74|       1708|  5000.0|  9|6/15/2021 0:00|6/24/2021 0:00| 7/6/2021 0:00|             979998|      1947|  ML81848993|     2021|     San Mateo|
|   92225|  Sold|2760 Colorado Riv...|    749900|    710000| 16|       2002| 19602.0|  2| 6/9/2021 0:00|6/11/2021 0:00|7/16/2021 0:00|             749900|      2005|CRSW21124462|     2

121510

In [ ]:
modify_df = joined_df.select("MLS Number", "Street Address", "zip_code", "county", "List Price", "Sale Price", "Age", "Sq Ft Total", "Lot Size","DOM","Year_Sale","Listing Date","Sale Date", "Year Built")
modify_df.show()
modify_df.count()

+------------+--------------------+--------+--------------+----------+----------+---+-----------+--------+---+---------+--------------+--------------+----------+
|  MLS Number|      Street Address|zip_code|        county|List Price|Sale Price|Age|Sq Ft Total|Lot Size|DOM|Year_Sale|  Listing Date|     Sale Date|Year Built|
+------------+--------------------+--------+--------------+----------+----------+---+-----------+--------+---+---------+--------------+--------------+----------+
|  ML81848993|    757 Douglas AVE |   94063|     San Mateo|    979998|   1350000| 74|       1708|  5000.0|  9|     2021|6/15/2021 0:00|6/24/2021 0:00|      1947|
|CRSW21124462|2760 Colorado Riv...|   92225|     Riverside|    749900|    710000| 16|       2002| 19602.0|  2|     2021| 6/9/2021 0:00|6/11/2021 0:00|      2005|
|CROC21137467|       680 Catalina |   92651|        Orange|   1185000|   1850000| 84|        998|  6970.0|  0|     2021|6/24/2021 0:00|6/24/2021 0:00|      1937|
|CRNP21120019|        2 S Po

121510

In [ ]:
from pyspark.ml.feature import StringIndexer

#OneHotEncoderEstimator

# create object of StringIndexer class and specify input and output column
SI_county = StringIndexer(inputCol='county',outputCol='County_Index')

# transform the data
modify_df = SI_county.fit(modify_df).transform(modify_df)

# view the transformed data
modify_df.select('county', 'County_Index').show(10)

modify_df.show()

+---------+------------+
|   county|County_Index|
+---------+------------+
|San Mateo|        11.0|
|Riverside|         3.0|
|   Orange|         1.0|
|   Orange|         1.0|
|   Orange|         1.0|
|   Orange|         1.0|
|   Orange|         1.0|
|   Orange|         1.0|
|   Orange|         1.0|
|   Orange|         1.0|
+---------+------------+
only showing top 10 rows

+------------+--------------------+--------+--------------+----------+----------+---+-----------+--------+---+---------+--------------+--------------+----------+------------+
|  MLS Number|      Street Address|zip_code|        county|List Price|Sale Price|Age|Sq Ft Total|Lot Size|DOM|Year_Sale|  Listing Date|     Sale Date|Year Built|County_Index|
+------------+--------------------+--------+--------------+----------+----------+---+-----------+--------+---+---------+--------------+--------------+----------+------------+
|  ML81848993|    757 Douglas AVE |   94063|     San Mateo|    979998|   1350000| 74|       1708|  

In [ ]:
### Show the df after adding the county index column
modify_df.show()

+------------+--------------------+--------+--------------+----------+----------+---+-----------+--------+---+---------+--------------+--------------+----------+------------+
|  MLS Number|      Street Address|zip_code|        county|List Price|Sale Price|Age|Sq Ft Total|Lot Size|DOM|Year_Sale|  Listing Date|     Sale Date|Year Built|County_Index|
+------------+--------------------+--------+--------------+----------+----------+---+-----------+--------+---+---------+--------------+--------------+----------+------------+
|  ML81848993|    757 Douglas AVE |   94063|     San Mateo|    979998|   1350000| 74|       1708|  5000.0|  9|     2021|6/15/2021 0:00|6/24/2021 0:00|      1947|        11.0|
|CRSW21124462|2760 Colorado Riv...|   92225|     Riverside|    749900|    710000| 16|       2002| 19602.0|  2|     2021| 6/9/2021 0:00|6/11/2021 0:00|      2005|         3.0|
|CROC21137467|       680 Catalina |   92651|        Orange|   1185000|   1850000| 84|        998|  6970.0|  0|     2021|6/24/

In [ ]:
# import sql function pyspark
import pyspark.sql.functions as f

# null values in each column
data_agg = modify_df.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in modify_df.columns])
data_agg.show()

+----------+--------------+--------+------+----------+----------+---+-----------+--------+---+---------+------------+---------+----------+------------+
|MLS Number|Street Address|zip_code|county|List Price|Sale Price|Age|Sq Ft Total|Lot Size|DOM|Year_Sale|Listing Date|Sale Date|Year Built|County_Index|
+----------+--------------+--------+------+----------+----------+---+-----------+--------+---+---------+------------+---------+----------+------------+
|         0|             0|       0|     0|         0|         0|  0|          0|       0|  0|        0|           0|        0|         0|           0|
+----------+--------------+--------+------+----------+----------+---+-----------+--------+---+---------+------------+---------+----------+------------+



In [ ]:
# Drop null values
final_df = modify_df.dropDuplicates(['MLS Number', 'zip_code', "Year_Sale", 'Sale Price', 'Listing Date'])
final_df.show()
final_df.count()

+------------+--------------------+--------+--------------+----------+----------+---+-----------+--------+---+---------+--------------+--------------+----------+------------+
|  MLS Number|      Street Address|zip_code|        county|List Price|Sale Price|Age|Sq Ft Total|Lot Size|DOM|Year_Sale|  Listing Date|     Sale Date|Year Built|County_Index|
+------------+--------------------+--------+--------------+----------+----------+---+-----------+--------+---+---------+--------------+--------------+----------+------------+
|CRPW21118939|  12930 Burgess AVE |   90638|   Los Angeles|    845000|    845000| 62|       1592|  6303.0| 13|     2021| 6/2/2021 0:00|6/15/2021 0:00|      1959|         0.0|
|CNSR21122752| 4916 Greenbush AVE |   91423|   Los Angeles|   1600000|   1710000| 70|       2475|  7206.0|  8|     2021|6/11/2021 0:00|6/19/2021 0:00|      1951|         0.0|
|CRNDP2106998|       6211 Agee ST |   92122|     San Diego|   1388000|   1428000| 53|       1698|  7100.0| 12|     2021|6/10/

4190

In [ ]:
final_df.na.drop(how='all')
final_df.show()
final_df.count()

+------------+--------------------+--------+--------------+----------+----------+---+-----------+--------+---+---------+--------------+--------------+----------+------------+
|  MLS Number|      Street Address|zip_code|        county|List Price|Sale Price|Age|Sq Ft Total|Lot Size|DOM|Year_Sale|  Listing Date|     Sale Date|Year Built|County_Index|
+------------+--------------------+--------+--------------+----------+----------+---+-----------+--------+---+---------+--------------+--------------+----------+------------+
|CRPW21118939|  12930 Burgess AVE |   90638|   Los Angeles|    845000|    845000| 62|       1592|  6303.0| 13|     2021| 6/2/2021 0:00|6/15/2021 0:00|      1959|         0.0|
|CNSR21122752| 4916 Greenbush AVE |   91423|   Los Angeles|   1600000|   1710000| 70|       2475|  7206.0|  8|     2021|6/11/2021 0:00|6/19/2021 0:00|      1951|         0.0|
|CRNDP2106998|       6211 Agee ST |   92122|     San Diego|   1388000|   1428000| 53|       1698|  7100.0| 12|     2021|6/10/

4190

In [ ]:
final_df.printSchema()

root
 |-- MLS Number: string (nullable = true)
 |-- Street Address: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- county: string (nullable = true)
 |-- List Price: integer (nullable = true)
 |-- Sale Price: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Sq Ft Total: integer (nullable = true)
 |-- Lot Size: double (nullable = true)
 |-- DOM: integer (nullable = true)
 |-- Year_Sale: integer (nullable = true)
 |-- Listing Date: string (nullable = true)
 |-- Sale Date: string (nullable = true)
 |-- Year Built: integer (nullable = true)
 |-- County_Index: double (nullable = false)



In [ ]:
final_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
MLS Number,4190,None,None,BA321030025,TM21-762052
Street Address,4190,None,None,"""1595 """"C"""" Josselyn Canyon RD """,9989 La Porte RD
zip_code,4190,93733.22840095466,1761.1977376478192,90002,96161
county,4190,None,None,Alameda,Yuba
List Price,4190,1038517.646778043,1041527.6312841626,75000,15868000
Sale Price,4190,1108018.570405728,1081943.3690121523,72000,15350000
Age,4190,46.71766109785203,26.72218545127131,0,152
Sq Ft Total,4190,1942.3334128878282,893.6931246817402,266,11853
Lot Size,4190,309681.166594272,1.1118406844002385E7,800.0,6.181164E8


In [ ]:
#import pandas
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
#from config import db_password

In [ ]:
# Get the columns and the rows that are not null.
final_df.dtypes

[('MLS Number', 'string'),
 ('Street Address', 'string'),
 ('zip_code', 'string'),
 ('county', 'string'),
 ('List Price', 'int'),
 ('Sale Price', 'int'),
 ('Age', 'int'),
 ('Sq Ft Total', 'int'),
 ('Lot Size', 'double'),
 ('DOM', 'int'),
 ('Year_Sale', 'int'),
 ('Listing Date', 'string'),
 ('Sale Date', 'string'),
 ('Year Built', 'int'),
 ('County_Index', 'double')]

In [ ]:
#Get all the Lot Size are greater than 800 in a new DataFrame.
final_df = final_df[final_df["Lot Size"] >= 800]
final_df.count()

4190

In [ ]:
# Create user dataframe to match active_user table
house_df = final_df.select('MLS Number', 'Street Address', 'zip_code', 'county', 'age', 'Sq Ft Total', 'Lot size', 'Year Built')
#clean_user_df = cleaned_df.select(["id","first_name","last_name","username"])
house_df.show()

+------------+--------------------+--------+--------------+---+-----------+--------+----------+
|  MLS Number|      Street Address|zip_code|        county|age|Sq Ft Total|Lot size|Year Built|
+------------+--------------------+--------+--------------+---+-----------+--------+----------+
|CRPW21118939|  12930 Burgess AVE |   90638|   Los Angeles| 62|       1592|  6303.0|      1959|
|CNSR21122752| 4916 Greenbush AVE |   91423|   Los Angeles| 70|       2475|  7206.0|      1951|
|CRNDP2106998|       6211 Agee ST |   92122|     San Diego| 53|       1698|  7100.0|      1968|
|CRPTP2104175|10615 GREENFORD D...|   92126|     San Diego| 42|       1900|  6400.0|      1979|
|CRIG21127301|16722 Desert Will...|   92394|San Bernardino| 14|       2607|  6249.0|      2007|
|CRIG21124407|      4064 Tamarind |   92530|     Riverside|  4|       1851|  6970.0|      2017|
|CROC21125012|   5252 Vineland DR |   92649|        Orange| 55|       2105|  6262.0|      1966|
|CRIG21111996| 3175 Timberline DR |   92

In [ ]:
# Create user dataframe to match billing_info table
sale_df = final_df.select("MLS Number", "zip_code", "county", "County_Index", "Year_Sale", "List Price", "Sale Price", "Age", "Sq Ft Total", "Lot Size","DOM","Sale Date")
sale_df.show()

+------------+--------+--------------+------------+---------+----------+----------+---+-----------+--------+---+--------------+
|  MLS Number|zip_code|        county|County_Index|Year_Sale|List Price|Sale Price|Age|Sq Ft Total|Lot Size|DOM|     Sale Date|
+------------+--------+--------------+------------+---------+----------+----------+---+-----------+--------+---+--------------+
|CRPW21118939|   90638|   Los Angeles|         0.0|     2021|    845000|    845000| 62|       1592|  6303.0| 13|6/15/2021 0:00|
|CNSR21122752|   91423|   Los Angeles|         0.0|     2021|   1600000|   1710000| 70|       2475|  7206.0|  8|6/19/2021 0:00|
|CRNDP2106998|   92122|     San Diego|         7.0|     2021|   1388000|   1428000| 53|       1698|  7100.0| 12|6/22/2021 0:00|
|CRPTP2104175|   92126|     San Diego|         7.0|     2021|    975000|    945000| 42|       1900|  6400.0| 19|6/29/2021 0:00|
|CRIG21127301|   92394|San Bernardino|         8.0|     2021|    379900|    417000| 14|       2607|  624

In [ ]:
price_df = final_df.select("MLS Number", "county", "County_Index", "Year_Sale", "List Price", "Sale Price", "DOM","Sale Date")
price_df.show()

+------------+--------------+------------+---------+----------+----------+---+--------------+
|  MLS Number|        county|County_Index|Year_Sale|List Price|Sale Price|DOM|     Sale Date|
+------------+--------------+------------+---------+----------+----------+---+--------------+
|CRPW21118939|   Los Angeles|         0.0|     2021|    845000|    845000| 13|6/15/2021 0:00|
|CNSR21122752|   Los Angeles|         0.0|     2021|   1600000|   1710000|  8|6/19/2021 0:00|
|CRNDP2106998|     San Diego|         7.0|     2021|   1388000|   1428000| 12|6/22/2021 0:00|
|CRPTP2104175|     San Diego|         7.0|     2021|    975000|    945000| 19|6/29/2021 0:00|
|CRIG21127301|San Bernardino|         8.0|     2021|    379900|    417000|  6|6/17/2021 0:00|
|CRIG21124407|     Riverside|         3.0|     2021|    555000|    555000|  6|6/15/2021 0:00|
|CROC21125012|        Orange|         1.0|     2021|    975000|   1040000| 20| 7/2/2021 0:00|
|CRIG21111996|     Riverside|         3.0|     2021|    6750

In [ ]:
machine_learning_df = final_df.select("zip_code", "county", "County_Index", "Year_Sale", "List Price", "Sale Price", "Age", "Sq Ft Total", "Lot Size","DOM", "Sale Date")
machine_learning_df.show()

+--------+--------------+------------+---------+----------+----------+---+-----------+--------+---+--------------+
|zip_code|        county|County_Index|Year_Sale|List Price|Sale Price|Age|Sq Ft Total|Lot Size|DOM|     Sale Date|
+--------+--------------+------------+---------+----------+----------+---+-----------+--------+---+--------------+
|   90638|   Los Angeles|         0.0|     2021|    845000|    845000| 62|       1592|  6303.0| 13|6/15/2021 0:00|
|   91423|   Los Angeles|         0.0|     2021|   1600000|   1710000| 70|       2475|  7206.0|  8|6/19/2021 0:00|
|   92122|     San Diego|         7.0|     2021|   1388000|   1428000| 53|       1698|  7100.0| 12|6/22/2021 0:00|
|   92126|     San Diego|         7.0|     2021|    975000|    945000| 42|       1900|  6400.0| 19|6/29/2021 0:00|
|   92394|San Bernardino|         8.0|     2021|    379900|    417000| 14|       2607|  6249.0|  6|6/17/2021 0:00|
|   92530|     Riverside|         3.0|     2021|    555000|    555000|  4|      

In [ ]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cvmp56um73d3.us-west-1.rds.amazonaws.com:5432/Final_Project_db"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write DataFrame to active_user table in RDS
house_df.write.jdbc(url=jdbc_url, table='house_info', mode=mode, properties=config)

In [ ]:
# Write dataframe to billing_info table in RDS
sale_df.write.jdbc(url=jdbc_url, table='sale_info', mode=mode, properties=config)

In [ ]:
# Write dataframe to payment_info table in RDS
price_df.write.jdbc(url=jdbc_url, table='price_info', mode=mode, properties=config)

In [ ]:
# Write dataframe to payment_info table in RDS
machine_learning_df.write.jdbc(url=jdbc_url, table='machine_learning', mode=mode, properties=config)

In [ ]:
# Create user dataframe to match billing_info table
final_data_df = final_df.select('MLS Number', 'Street Address', 'zip_code', 'county', 'County_Index', "Year_Sale", 'List Price', 'Sale Price', 'Age', 'Sq Ft Total', 'Lot Size', 'DOM', 'Listing Date', 'Sale Date', 'Year Built')
final_data_df.show()

+------------+--------------------+--------+--------------+------------+---------+----------+----------+---+-----------+--------+---+--------------+--------------+----------+
|  MLS Number|      Street Address|zip_code|        county|County_Index|Year_Sale|List Price|Sale Price|Age|Sq Ft Total|Lot Size|DOM|  Listing Date|     Sale Date|Year Built|
+------------+--------------------+--------+--------------+------------+---------+----------+----------+---+-----------+--------+---+--------------+--------------+----------+
|CRPW21118939|  12930 Burgess AVE |   90638|   Los Angeles|         0.0|     2021|    845000|    845000| 62|       1592|  6303.0| 13| 6/2/2021 0:00|6/15/2021 0:00|      1959|
|CNSR21122752| 4916 Greenbush AVE |   91423|   Los Angeles|         0.0|     2021|   1600000|   1710000| 70|       2475|  7206.0|  8|6/11/2021 0:00|6/19/2021 0:00|      1951|
|CRNDP2106998|       6211 Agee ST |   92122|     San Diego|         7.0|     2021|   1388000|   1428000| 53|       1698|  710

In [ ]:
# Write dataframe to payment_info table in RDS
final_data_df.write.jdbc(url=jdbc_url, table='final_data', mode=mode, properties=config)